In [2]:
import os
import glob
import pandas as pd
import itertools

import Bio
from Bio import Entrez, SeqIO
from Bio import AlignIO

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
#path to bam in csv format
path_to_bam_file = '../Results_fragseq_rep1/Fraq_Seq_S16_S16_R1_001_R2_combi_ref_proper_20_40.csv'
#path to output df
df_align = '../Results_fragseq_rep1/S16_S16_20_40_align_df.csv'



+ open a converted to csv bam table

In [20]:
bbbb = pd.read_csv(path_to_bam_file,  sep =' ') #, header=None)
bbbb.shape[0]

415524

In [6]:
bbbb.head(100)

,qname,flag,rname,strand,pos,qwidth,mapq,cigar,mrnm,mpos,isize,seq,qual
1,A01492:10:HNH2FDMXX:1:1101:10086:11569,163,30,+,1,30,60,30M,30,1,30,TGTCAGTGACTGCGCCAAAACATCTTCGGA,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
2,A01492:10:HNH2FDMXX:1:1102:29948:28338,163,30,+,1,30,60,30M,30,1,30,TGTCAGTGACTGCGCCAAAACATCTTCGGA,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
3,A01492:10:HNH2FDMXX:1:1115:22092:3364,163,30,+,1,28,60,28M,30,1,28,TGTCAGTGACTGCGCCAAAACATCTTCG,FFFFFFFFFFFFFFFFFFF:FFFFFFFF
4,A01492:10:HNH2FDMXX:1:1125:21016:24549,99,30,+,1,30,60,30M,30,1,30,TGTCAGTGACTGCGCCAAAACATCTTCGGA,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
5,A01492:10:HNH2FDMXX:1:1136:28103:35383,163,30,+,1,30,60,30M,30,1,30,TGTCAGTGACTGCGCCAAAACATCTTCGGA,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,A01492:10:HNH2FDMXX:2:1122:3658:34695,163,30,+,1,30,60,30M,30,1,30,TGTCAGTGACTGCGCCAAAACATCTTCGGA,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
97,A01492:10:HNH2FDMXX:2:1132:16975:22185,99,30,+,1,30,60,30M,30,1,30,TGTCAGTGACTGCGCCAAAACATCTTCGGA,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
98,A01492:10:HNH2FDMXX:2:1133:13440:24017,163,30,+,1,30,60,30M,30,1,30,TGTCAGTGACTGCGCCAAAACATCTTCGGA,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
99,A01492:10:HNH2FDMXX:2:1137:29821:26365,163,30,+,1,30,60,30M,30,1,30,TGTCAGTGACTGCGCCAAAACATCTTCGGA,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF


+ rename columns and moderate them

In [400]:
bbbb.rename(columns = {'qname' : 'ID', 'mrnm' : 'DNA', 'mpos' : 'left', 'isize' : 'width'}, inplace = True)
#bbbb = bbbb.drop(columns = ['pos','qwidth', 'mapq', 'seq','qual'], axis = 1)

In [416]:
bbbb

,ID,flag,rname,cigar,DNA,left,width,seq,qual,right
0,K00370:89:HKTHLBBXY:2:1111:14864:20814,83,BL21_AI_CP047231,32M,BL21_AI_CP047231,1,32,AGCTTTTCATTCTGACTGCAACGGGCAATATG,JJJJJJJJJJJJJJJJJJJJJJJJJJJFFFAA,32
1,K00370:89:HKTHLBBXY:2:1120:16234:8172,83,BL21_AI_CP047231,29M,BL21_AI_CP047231,1,29,AGCTTTTCATTCTGACTGCAACGGGCAAT,JJFFJJJFJAFAAJFJFJJJJAJJFFFA<,29
2,K00370:89:HKTHLBBXY:2:1218:10531:31839,83,BL21_AI_CP047231,31M,BL21_AI_CP047231,1,31,AGCTTTTCATTCTGACTGCAACGGGCAATAT,JJJJJJJJJJJJJJJJJJJJJJJJJJFFFAA,31
3,K00370:89:HKTHLBBXY:2:1218:9201:33721,83,BL21_AI_CP047231,31M,BL21_AI_CP047231,1,31,AGCTTTTCATTCTGACTGCAACGGGCAATAT,AJJJJJJJJ<AJJJJJJJJJJJJJFJF<AAA,31
4,K00370:89:HKTHLBBXY:2:2107:20659:44429,83,BL21_AI_CP047231,55M,BL21_AI_CP047231,1,55,AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATT...,JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...,55
...,...,...,...,...,...,...,...,...,...,...
1239992,K00370:89:HKTHLBBXY:2:1103:8897:14590,83,BL21_AI_CP047231,46M,BL21_AI_CP047231,4530519,46,TGAAAAAAATATCACCAAATAAAAAACGCCTTAGTAAGTATTTTTC,JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ,4530564
1239993,K00370:89:HKTHLBBXY:2:2222:4046:48843,83,BL21_AI_CP047231,46M,BL21_AI_CP047231,4530519,46,TGAAAAAAATATCACCAAATAAAAAACGCCTTAGTAAGTATTTTTC,JJJJJJJJJJJJJJJJJJJJJJJJFAJJJJJJJJJJJJJJFJFFAJ,4530564
1239994,K00370:89:HKTHLBBXY:2:2128:6421:41598,83,BL21_AI_CP047231,40M,BL21_AI_CP047231,4530525,40,AAATATCACCAAATAAAAAACGCCTTAGTAAGTATTTTTC,JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ,4530564
1239995,K00370:89:HKTHLBBXY:2:2207:18578:48667,83,BL21_AI_CP047231,38M,BL21_AI_CP047231,4530522,43,ATATCACCAAATAAAAAACGCCTTAGTAAGTATTTTTC,JFJJJJJJJJJJJJJJFA-JJJJJJJJJJJJFJJFJJJ,4530564


In [402]:
bbbb = bbbb.loc[bbbb['DNA'] == "BL21_AI_CP047231"]

In [404]:
len("AAATATCACCAAATAAAAAACGCCTTAGTAAGTATTTTTC")

40

In [407]:
bbbb.to_csv(df_intermed_path, index=False)

In [408]:
bbbb = pd.read_csv(df_intermed_path)

In [409]:
bbbb = bbbb.drop(columns = ['pos','qwidth', 'mapq'], axis = 1)

In [415]:
bbbb = bbbb.drop(columns = ['strand'], axis = 1)

In [417]:
bbbb1 = bbbb.loc[bbbb['left'] == 850482]
bbbb1

,ID,flag,rname,cigar,DNA,left,width,seq,qual,right
216407,K00370:89:HKTHLBBXY:2:2117:15493:39137,83,BL21_AI_CP047231,79M,BL21_AI_CP047231,850482,73,GTGAAACTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCAT...,<-JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...,850554
216408,K00370:89:HKTHLBBXY:2:2211:19004:8682,83,BL21_AI_CP047231,72M,BL21_AI_CP047231,850482,66,TAGAAGCTGAGTTACNGTCTGTTTTCCTTGTTGGAACGGAGAGCAT...,-JJJJJJJJJJJJJJ#JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...,850547
216409,K00370:89:HKTHLBBXY:2:2216:23784:28639,83,BL21_AI_CP047231,57M,BL21_AI_CP047231,850482,51,TAGAAGCTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCAT...,-JJJJJJJJJJJJFJFJFFJJJFJJJJFJJJJJJFFJJJJJJJJJJ...,850532
216415,K00370:89:HKTHLBBXY:2:2220:20273:13166,83,BL21_AI_CP047231,57M,BL21_AI_CP047231,850482,52,NGAGTCTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATC...,#JF<-JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...,850533
216416,K00370:89:HKTHLBBXY:2:2226:12834:43251,83,BL21_AI_CP047231,76M,BL21_AI_CP047231,850482,71,TGAAGCTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATC...,<JJJJJJJF<JJJJJJJJJJJJJJFJJJJJJFFJJJJJJJJJJJJJ...,850552
...,...,...,...,...,...,...,...,...,...,...
259333,K00370:89:HKTHLBBXY:2:2212:18842:3231,83,BL21_AI_CP047231,51M,BL21_AI_CP047231,850482,70,CCTTGTTGGCCCGGAGAGCATCGCCTGATGCTCTCCGAGCCAACCA...,JJJJJJJJJAAJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJAJJJ...,850551
259337,K00370:89:HKTHLBBXY:2:2210:19492:40438,83,BL21_AI_CP047231,56M,BL21_AI_CP047231,850482,76,CTTGTTGGAACGGAGAGCATCGCCTGATGCTCTCCGAGCCAACCAG...,JJJJFJJJJJJJJJJJJJJJJJJJJJJJJJFJJJJJJFJFAFJJJJ...,850557
259338,K00370:89:HKTHLBBXY:2:2212:22029:3934,83,BL21_AI_CP047231,51M,BL21_AI_CP047231,850482,71,CTTGTTGGACCGGAGAGCATCGCCTGATGCTCTCCGAGCCAACCAG...,JJJJJJJJJFFJJJJJJJJJJJJJJJJJJJJJJFJJJJJJJJJJJJ...,850552
259339,K00370:89:HKTHLBBXY:2:2212:22861:3934,83,BL21_AI_CP047231,51M,BL21_AI_CP047231,850482,71,CTTGTTGGACCGGAGAGCATCGCCTGATGCTCTCCGAGCCAACCAG...,JJJJJJJJFAAJJJJJJJJJJJJJJJJFJJJJJJJJJJJJJJJJJJ...,850552


In [418]:
bbbb2 = bbbb1.loc[bbbb['right'] == 850566]
bbbb2

,ID,flag,rname,cigar,DNA,left,width,seq,qual,right
217300,K00370:89:HKTHLBBXY:2:1102:12926:8770,83,BL21_AI_CP047231,85M,BL21_AI_CP047231,850482,85,CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTG...,JJJJJJJJJJJJJJJFJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...,850566
218196,K00370:89:HKTHLBBXY:2:1104:24180:20568,83,BL21_AI_CP047231,85M,BL21_AI_CP047231,850482,85,CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTG...,JJJJFFAJFJJFFJJJJJJFJJFJJJJJJFFJJJJJJJJFJFJJJJ...,850566
220922,K00370:89:HKTHLBBXY:2:1111:3427:42987,83,BL21_AI_CP047231,85M,BL21_AI_CP047231,850482,85,CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTG...,JJJJJJJJJJJJJJJJJJFJJJJJJJJJJJJJJJFJJJJJJJFJJJ...,850566
221962,K00370:89:HKTHLBBXY:2:1114:8511:33299,83,BL21_AI_CP047231,85M,BL21_AI_CP047231,850482,85,CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTG...,<FFFJJJJJJFJFJJJJJJ7JJJJJJJFFAJJJF7JJ<JJAJJFJF...,850566
226368,K00370:89:HKTHLBBXY:2:1127:21054:4391,83,BL21_AI_CP047231,85M,BL21_AI_CP047231,850482,85,CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTG...,JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...,850566
227383,K00370:89:HKTHLBBXY:2:1202:25570:2861,83,BL21_AI_CP047231,85M,BL21_AI_CP047231,850482,85,CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTG...,JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...,850566
229634,K00370:89:HKTHLBBXY:2:1208:26250:15434,83,BL21_AI_CP047231,85M,BL21_AI_CP047231,850482,85,CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTG...,JJJJJJJJJJJJJJJJJJJJJJJJJJJJJFJJJJJJJJJJJJJJJJ...,850566
230746,K00370:89:HKTHLBBXY:2:1211:23876:17509,83,BL21_AI_CP047231,85M,BL21_AI_CP047231,850482,85,CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTG...,JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...,850566
232715,K00370:89:HKTHLBBXY:2:1216:13078:42829,83,BL21_AI_CP047231,85M,BL21_AI_CP047231,850482,85,CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTG...,JJJJJJAFJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJFJJJJJJ...,850566
233172,K00370:89:HKTHLBBXY:2:1218:6076:1261,83,BL21_AI_CP047231,85M,BL21_AI_CP047231,850482,85,CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTG...,JFJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...,850566


In [427]:
bbbb2.shape

(43, 10)

In [419]:
bbbb2["seq"][217300]

'CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTGATGCTCTCCGAGCCAACCAGGAAACCCGTTTTTTCTGAC'

In [429]:
#5 конец
ref="gtcagaaaaaacgggtttcctggttggctcggagagcatcaggcgatgctctccgttccaacaaggaaaacagacagtaactcag" #лево тут,3конец!!
#убрала букву а на конце,тк она не входит в ретрон

In [421]:
>>> from Bio.Seq import Seq
#>>> from Bio.Alphabet import IUPAC
>>> my_dna = Seq("CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTGATGCTCTCCGAGCCAACCAGGAAACCCGTTTTTTCTGAC")
>>> my_dna
#Seq('CCCCCGATAGNR')
>>> pupa=my_dna.reverse_complement()
#Seq('YNCTATCGGGGG')

In [423]:
str(pupa)

'GTCAGAAAAAACGGGTTTCCTGGTTGGCTCGGAGAGCATCAGGCGATGCTCTCCGTTCCAACAAGGAAAACAGACAGTAACTCAG'

In [430]:
len(ref)

85

In [428]:
len(pupa) # это наш ретрончик ыыыы

85

In [456]:
bbbb53.shape

(2976, 10)

In [431]:
bbbb1.shape

(40680, 10)

In [455]:
bbbb35=bbbb1.loc[bbbb['right'] == 850535]
bbbb35

,ID,flag,rname,cigar,DNA,left,width,seq,qual,right
216556,K00370:89:HKTHLBBXY:2:2211:29254:3231,83,BL21_AI_CP047231,58M,BL21_AI_CP047231,850482,54,GNAACTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCAACG...,J#JJJJJJJFAJJJJJJJJJJJJJJJJJJJFA<JJJJJJJJFA7JJ...,850535
216557,K00370:89:HKTHLBBXY:2:2211:4645:5464,83,BL21_AI_CP047231,58M,BL21_AI_CP047231,850482,54,GNAGCTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCG...,F#FJFFJJJJFAFFFJFAF7F-JFJFJ<7JJJFA7JAJJJAJJF7J...,850535
216560,K00370:89:HKTHLBBXY:2:2211:9567:34952,83,BL21_AI_CP047231,58M,BL21_AI_CP047231,850482,54,GNAGCTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCG...,J#JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...,850535
216562,K00370:89:HKTHLBBXY:2:2211:5020:46170,83,BL21_AI_CP047231,58M,BL21_AI_CP047231,850482,54,GNAGCTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCG...,J#JJJJJJJJJJJJJJJJJFJJJJJJJJJJJJJFJJJJJJJJJFJJ...,850535
216582,K00370:89:HKTHLBBXY:2:2220:25550:40913,83,BL21_AI_CP047231,58M,BL21_AI_CP047231,850482,54,GNAGCTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCG...,J#JJJJJJJJJJJJJJJJJJJJFFJJJJJJJJJFJJJJJJJJJJJJ...,850535
...,...,...,...,...,...,...,...,...,...,...
259069,K00370:89:HKTHLBBXY:2:2217:13768:22362,83,BL21_AI_CP047231,49M,BL21_AI_CP047231,850482,54,TTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTGATGCTCTC,JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJFFFFAA,850535
259224,K00370:89:HKTHLBBXY:2:2209:20577:39541,83,BL21_AI_CP047231,46M,BL21_AI_CP047231,850482,54,CTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTGATGCTCTC,JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJFFFAA,850535
259225,K00370:89:HKTHLBBXY:2:2209:28828:39945,83,BL21_AI_CP047231,46M,BL21_AI_CP047231,850482,54,CTGTCTGTTTTCCTTGTTGGAACGGCGAGCATCGCCTGATGCTCTC,JJJJJJJJJJJJJJJJJJJJJJJJFJJJJJJJJJJJJJJJJFFFAA,850535
259226,K00370:89:HKTHLBBXY:2:2209:27762:40877,83,BL21_AI_CP047231,46M,BL21_AI_CP047231,850482,54,CTGTCTGTTTTCCTTGTTGGAACGGCGAGCATCGCCTGATGCTCTC,JJFJFJJJJJJJJJJ<<FA7FAJJAJFJJJJJ<JJFJFJJFFFAAA,850535


In [459]:
bbbb35.head(20)

,ID,flag,rname,cigar,DNA,left,width,seq,qual,right
216556,K00370:89:HKTHLBBXY:2:2211:29254:3231,83,BL21_AI_CP047231,58M,BL21_AI_CP047231,850482,54,GNAACTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCAACG...,J#JJJJJJJFAJJJJJJJJJJJJJJJJJJJFA<JJJJJJJJFA7JJ...,850535
216557,K00370:89:HKTHLBBXY:2:2211:4645:5464,83,BL21_AI_CP047231,58M,BL21_AI_CP047231,850482,54,GNAGCTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCG...,F#FJFFJJJJFAFFFJFAF7F-JFJFJ<7JJJFA7JAJJJAJJF7J...,850535
216560,K00370:89:HKTHLBBXY:2:2211:9567:34952,83,BL21_AI_CP047231,58M,BL21_AI_CP047231,850482,54,GNAGCTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCG...,J#JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...,850535
216562,K00370:89:HKTHLBBXY:2:2211:5020:46170,83,BL21_AI_CP047231,58M,BL21_AI_CP047231,850482,54,GNAGCTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCG...,J#JJJJJJJJJJJJJJJJJFJJJJJJJJJJJJJFJJJJJJJJJFJJ...,850535
216582,K00370:89:HKTHLBBXY:2:2220:25550:40913,83,BL21_AI_CP047231,58M,BL21_AI_CP047231,850482,54,GNAGCTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCG...,J#JJJJJJJJJJJJJJJJJJJJFFJJJJJJJJJFJJJJJJJJJJJJ...,850535
216724,K00370:89:HKTHLBBXY:2:2211:20740:40948,83,BL21_AI_CP047231,57M,BL21_AI_CP047231,850482,54,NAGCTGAGTCACTGTCTGTTTTCCTCGTTGGAACGGAGAGCATCGC...,#JJJJJJF<<JJJJJJJJFJJJJF<7JJJJJJFJJJJJJJJJJJJJ...,850535
216927,K00370:89:HKTHLBBXY:2:1101:22881:4110,83,BL21_AI_CP047231,54M,BL21_AI_CP047231,850482,54,CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTG...,FJJJJJJJJJJJ<JJJJFJFFJJJJJJJFJA<JJJJJAJF7FFJ7F...,850535
216937,K00370:89:HKTHLBBXY:2:1101:27285:6255,83,BL21_AI_CP047231,54M,BL21_AI_CP047231,850482,54,CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTG...,JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJFJJJJJFJJ...,850535
216938,K00370:89:HKTHLBBXY:2:1101:26880:6325,83,BL21_AI_CP047231,54M,BL21_AI_CP047231,850482,54,CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTG...,JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...,850535
216981,K00370:89:HKTHLBBXY:2:1101:23328:16524,83,BL21_AI_CP047231,54M,BL21_AI_CP047231,850482,54,CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTG...,JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...,850535


In [457]:
bbbb35["seq"][216556]

'GNAACTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCAACGCCTGATGCTCTC'

In [460]:
bbbb35["seq"][217073]

'CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTGATGCTCTC'

In [461]:
okok = Seq("CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTGATGCTCTC")
okok_rev=okok.reverse_complement()
okolupa=str(okok_rev)
okolupa

'GAGAGCATCAGGCGATGCTCTCCGTTCCAACAAGGAAAACAGACAGTAACTCAG'

In [462]:
bbbb44=bbbb1.loc[bbbb['right'] == 850544]
bbbb44

,ID,flag,rname,cigar,DNA,left,width,seq,qual,right
216781,K00370:89:HKTHLBBXY:2:1203:31456:27619,83,BL21_AI_CP047231,65M,BL21_AI_CP047231,850482,63,ATCTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCC...,7-JJFFFF7-<JJAFJFJF<JA-<<7FJF--FJJFFJF-FAFF-JJ...,850544
216965,K00370:89:HKTHLBBXY:2:1101:4148:13675,83,BL21_AI_CP047231,63M,BL21_AI_CP047231,850482,63,CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTG...,FJJJJJJJJJJFFJJJJJJJJJJJJJJJJFFJJAJJJJFJJJJJJJ...,850544
216977,K00370:89:HKTHLBBXY:2:1101:12642:15662,83,BL21_AI_CP047231,63M,BL21_AI_CP047231,850482,63,CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTG...,JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...,850544
217000,K00370:89:HKTHLBBXY:2:1101:5680:19566,83,BL21_AI_CP047231,63M,BL21_AI_CP047231,850482,63,CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTG...,JJJJJJJJJJJJJJJJFJJJJAJJJJJJJJJJJJJJJJJFJJJJJF...,850544
217011,K00370:89:HKTHLBBXY:2:1101:7669:21149,83,BL21_AI_CP047231,63M,BL21_AI_CP047231,850482,63,CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTG...,JJJJJJJJJJJJJJJJJJFJJJJJJJJJJJJJJJJJJJJFJJJJJJ...,850544
...,...,...,...,...,...,...,...,...,...,...
259185,K00370:89:HKTHLBBXY:2:2210:25652:46997,83,BL21_AI_CP047231,56M,BL21_AI_CP047231,850482,63,ACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTGATGCTCT...,FJFJFJJFAJJFA<JFJJFFFA7<FJJJJFFA-<FJJJFJFAJAJ<...,850544
259196,K00370:89:HKTHLBBXY:2:2220:25327:34758,83,BL21_AI_CP047231,56M,BL21_AI_CP047231,850482,63,ACTGTCTGTTTTCCTTGTTGGTACGGAGAGCATCGCCTGATGCTCT...,JJF-JJJJJJJJJJJJJJJF7-JJJJJJJJJJJJJJJJJJJJJJJJ...,850544
259235,K00370:89:HKTHLBBXY:2:2219:27945:32332,83,BL21_AI_CP047231,55M,BL21_AI_CP047231,850482,63,CTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTGATGCTCTC...,JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...,850544
259236,K00370:89:HKTHLBBXY:2:2219:28077:39277,83,BL21_AI_CP047231,55M,BL21_AI_CP047231,850482,63,CTGTCTGTTTTCCTCGTTGGAACGGAGAGCATCGCCTGATGCTCTC...,JJJJJJJJJJJJF<7JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...,850544


In [463]:
bbbb44["seq"][216977]

'CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTGATGCTCTCCGAGCCAAC'

In [464]:
okok = Seq("CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTGATGCTCTCCGAGCCAAC")
okok_rev=okok.reverse_complement()
okolupa=str(okok_rev)
okolupa

'GTTGGCTCGGAGAGCATCAGGCGATGCTCTCCGTTCCAACAAGGAAAACAGACAGTAACTCAG'

In [433]:
ok_seq = bbbb53["seq"][216930]
ok_seq

'CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTGATGCTCTCCGAGCCAACCAGGAAACC'

In [434]:
neok_seq=bbbb53["seq"][259353]
neok_seq

'TTGTTGGAACCGAGAGCATCGCCTGATGCTCTCCGAGCCAACCAGGAAACC'

In [449]:
okok = Seq("CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTGATGCTCTCCGAGCCAACCAGGAAACC")
okok_rev=okok.reverse_complement()
okolupa=str(okok_rev)
okolupa

'GGTTTCCTGGTTGGCTCGGAGAGCATCAGGCGATGCTCTCCGTTCCAACAAGGAAAACAGACAGTAACTCAG'

In [438]:
neokok = Seq("TTGTTGGAACCGAGAGCATCGCCTGATGCTCTCCGAGCCAACCAGGAAACC")
neokok_rev=neokok.reverse_complement()
neokolupa=str(neokok_rev)
neokolupa

'GGTTTCCTGGTTGGCTCGGAGAGCATCAGGCGATGCTCTCGGTTCCAACAA'

In [ ]:
#K00370:89:HKTHLBBXY:2:2212:22627:9860

In [440]:
name_file="sel_all_flag163.csv"
path_to_bam_file = "/Volumes/Transcend/coverage_counts/map-3-5-primes/" + folder + name_file
gggg = pd.read_csv(path_to_bam_file,  sep =' ') #, header=None)

In [444]:
gggg

,qname,flag,rname,strand,pos,qwidth,mapq,cigar,mrnm,mpos,isize,seq,qual
1,K00370:89:HKTHLBBXY:2:1105:24870:30802,163,20,+,1,20,40,20M,20,1,20,CAATCATAGAGTACGGTAGG,FFJJJJJJJJJJJJJJJJJJ
2,K00370:89:HKTHLBBXY:2:1207:26058:28358,163,20,+,1,20,40,20M,20,1,20,CAATCATAGAGTACGGTAGG,JJJJJJJJJJJJJJJJJJJJ
3,K00370:89:HKTHLBBXY:2:1217:27539:11618,163,20,+,1,20,40,20M,20,1,20,CAATCATAGAGTACGGTAGG,FFJJJJJJFJJJJFJJFJJJ
4,K00370:89:HKTHLBBXY:2:1225:22130:41317,163,20,+,1,20,40,20M,20,1,20,CAATCATAGAGTACGGTAGG,JJJJJJJJJJJJJJJJJJJJ
5,K00370:89:HKTHLBBXY:2:2101:13088:47665,163,20,+,1,20,40,20M,20,1,20,CAATCATAGAGTACGGTAGG,JJJJJJJJJJJJJJJJJJJJ
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389491,K00370:89:HKTHLBBXY:2:1106:9019:2246,163,pCDF_Ec.cas1/2,+,4714,21,46,21M,pCDF_Ec.cas1/2,4714,21,GCGACTCCTGCATTAGGAAAT,FFFJJJJJJJJJJJJJJJJJJ
1389492,K00370:89:HKTHLBBXY:2:1205:3823:21025,163,pCDF_Ec.cas1/2,+,4714,21,46,21M,pCDF_Ec.cas1/2,4714,21,GCGACTCCTGCATTAGGAAAT,FFJJJJJJJJJJJJJJJFJJJ
1389493,K00370:89:HKTHLBBXY:2:1218:27823:17597,163,pCDF_Ec.cas1/2,+,4714,21,46,21M,pCDF_Ec.cas1/2,4714,21,GCGACTCCTGCATTAGGAAAT,JJJJJJJJJJJFJJJJJJJJJ
1389494,K00370:89:HKTHLBBXY:2:1225:24282:35268,163,pCDF_Ec.cas1/2,+,4714,21,46,21M,pCDF_Ec.cas1/2,4714,21,GCGACTCCTGCATTAGGAAAT,JJJJJJJJJJJJJJJJJJJJJ


In [445]:
for i in range(1,gggg.shape[0]+1):
    if gggg["qname"][i]=="K00370:89:HKTHLBBXY:2:2212:22627:9860":
        print(gggg["cigar"][i])
        print(gggg["isize"][i])
        print(gggg["seq"][i])

72M
72
CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTGATGCTCTCCGAGCCAACCAGGAAACC


In [446]:
neokok = Seq("CTGAGTTACTGTCTGTTTTCCTTGTTGGAACGGAGAGCATCGCCTGATGCTCTCCGAGCCAACCAGGAAACC")
neokok_rev=neokok.reverse_complement()
neokolupa=str(neokok_rev)
neokolupa

'GGTTTCCTGGTTGGCTCGGAGAGCATCAGGCGATGCTCTCCGTTCCAACAAGGAAAACAGACAGTAACTCAG'

In [447]:
len(neokolupa)

72

In [451]:
len(okolupa)

72

In [452]:
okolupa

'GGTTTCCTGGTTGGCTCGGAGAGCATCAGGCGATGCTCTCCGTTCCAACAAGGAAAACAGACAGTAACTCAG'

In [453]:
ref

'gtcagaaaaaacgggtttcctggttggctcggagagcatcaggcgatgctctccgttccaacaaggaaaacagacagtaactcag'

In [ ]:
"gtcagaaaaaacg" + "ggtttcctggttggctcggagagcatcaggcgatgctctccgttccaacaaggaaaacagacagtaactcag"

In [7]:
bbbb.rename(columns = {'qname' : 'ID', 'mrnm' : 'DNA', 'mpos' : 'left', 'isize' : 'width'}, inplace = True)
bbbb = bbbb.drop(columns = ['pos','qwidth', 'mapq', 'seq','qual'], axis = 1)


In [8]:
bbbb = bbbb.drop(columns = ['strand'], axis = 1)

In [9]:
bbbb.shape

(415524, 7)

In [10]:
bbbb

,ID,flag,rname,cigar,DNA,left,width
1,A01492:10:HNH2FDMXX:1:1101:10086:11569,163,30,30M,30,1,30
2,A01492:10:HNH2FDMXX:1:1102:29948:28338,163,30,30M,30,1,30
3,A01492:10:HNH2FDMXX:1:1115:22092:3364,163,30,28M,30,1,28
4,A01492:10:HNH2FDMXX:1:1125:21016:24549,99,30,30M,30,1,30
5,A01492:10:HNH2FDMXX:1:1136:28103:35383,163,30,30M,30,1,30
...,...,...,...,...,...,...,...
415520,A01492:10:HNH2FDMXX:2:1110:3956:14826,83,pCDF_Ec.cas1/2,21M,pCDF_Ec.cas1/2,4714,-21
415521,A01492:10:HNH2FDMXX:1:2347:29559:3176,163,pCDF_Ec.cas1/2,20M,pCDF_Ec.cas1/2,4715,20
415522,A01492:10:HNH2FDMXX:2:2145:18376:12305,99,pCDF_Ec.cas1/2,20M,pCDF_Ec.cas1/2,4715,20
415523,A01492:10:HNH2FDMXX:1:2347:29559:3176,83,pCDF_Ec.cas1/2,20M,pCDF_Ec.cas1/2,4715,-20


In [11]:
def module_val(s):
    return abs(s)

def plus_minus(s):
    if s == 99 or s == 147:
        return "dir"
    elif s == 83 or s == 163:
        return "rev"

In [12]:
bbbb['flag'] = bbbb['flag'].apply(plus_minus)
bbbb['width'] = bbbb['width'].apply(module_val)
bbbb['right'] = bbbb['left'] + bbbb['width'] - 1
bbbb

,ID,flag,rname,cigar,DNA,left,width,right
1,A01492:10:HNH2FDMXX:1:1101:10086:11569,rev,30,30M,30,1,30,30
2,A01492:10:HNH2FDMXX:1:1102:29948:28338,rev,30,30M,30,1,30,30
3,A01492:10:HNH2FDMXX:1:1115:22092:3364,rev,30,28M,30,1,28,28
4,A01492:10:HNH2FDMXX:1:1125:21016:24549,dir,30,30M,30,1,30,30
5,A01492:10:HNH2FDMXX:1:1136:28103:35383,rev,30,30M,30,1,30,30
...,...,...,...,...,...,...,...,...
415520,A01492:10:HNH2FDMXX:2:1110:3956:14826,rev,pCDF_Ec.cas1/2,21M,pCDF_Ec.cas1/2,4714,21,4734
415521,A01492:10:HNH2FDMXX:1:2347:29559:3176,rev,pCDF_Ec.cas1/2,20M,pCDF_Ec.cas1/2,4715,20,4734
415522,A01492:10:HNH2FDMXX:2:2145:18376:12305,dir,pCDF_Ec.cas1/2,20M,pCDF_Ec.cas1/2,4715,20,4734
415523,A01492:10:HNH2FDMXX:1:2347:29559:3176,rev,pCDF_Ec.cas1/2,20M,pCDF_Ec.cas1/2,4715,20,4734


In [13]:
bbbb = bbbb.drop(columns = ['rname', 'cigar'], axis = 1)
bbbb.rename(columns = {'width' : 'isize'}, inplace = True)
bbbb['width'] = bbbb['isize']
bbbb = bbbb.drop(columns = ['isize'], axis = 1)
bbbb.rename(columns = {'flag' : 'strand'}, inplace = True)

bbbb

,ID,strand,DNA,left,right,width
1,A01492:10:HNH2FDMXX:1:1101:10086:11569,rev,30,1,30,30
2,A01492:10:HNH2FDMXX:1:1102:29948:28338,rev,30,1,30,30
3,A01492:10:HNH2FDMXX:1:1115:22092:3364,rev,30,1,28,28
4,A01492:10:HNH2FDMXX:1:1125:21016:24549,dir,30,1,30,30
5,A01492:10:HNH2FDMXX:1:1136:28103:35383,rev,30,1,30,30
...,...,...,...,...,...,...
415520,A01492:10:HNH2FDMXX:2:1110:3956:14826,rev,pCDF_Ec.cas1/2,4714,4734,21
415521,A01492:10:HNH2FDMXX:1:2347:29559:3176,rev,pCDF_Ec.cas1/2,4715,4734,20
415522,A01492:10:HNH2FDMXX:2:2145:18376:12305,dir,pCDF_Ec.cas1/2,4715,4734,20
415523,A01492:10:HNH2FDMXX:1:2347:29559:3176,rev,pCDF_Ec.cas1/2,4715,4734,20


+ save the dataframe or before that choose the reference (see below)

In [14]:
#df_align = '/Users/unnost/workwork/prespacers_task/Results_fragseq/Oct14_align_df.csv'
bbbb.to_csv(df_align, index=False)

+ here we can possibly choose a particular reference

In [15]:
bbbb = bbbb.loc[bbbb['DNA'] == "BL21_AI_CP047231"]

In [16]:
bbbb

,ID,strand,DNA,left,right,width
28509,A01492:10:HNH2FDMXX:1:2160:30092:22013,rev,BL21_AI_CP047231,1,34,34
28510,A01492:10:HNH2FDMXX:1:2374:3477:11490,dir,BL21_AI_CP047231,1,24,24
28511,A01492:10:HNH2FDMXX:1:2374:1262:13980,dir,BL21_AI_CP047231,1,24,24
28512,A01492:10:HNH2FDMXX:1:2443:12436:25786,dir,BL21_AI_CP047231,1,37,37
28513,A01492:10:HNH2FDMXX:1:2450:19379:11036,rev,BL21_AI_CP047231,1,24,24
...,...,...,...,...,...,...
377070,A01492:10:HNH2FDMXX:2:2141:25310:34209,dir,BL21_AI_CP047231,4530535,4530564,30
377071,A01492:10:HNH2FDMXX:1:2470:1994:9142,dir,BL21_AI_CP047231,4530535,4530564,30
377072,A01492:10:HNH2FDMXX:2:2141:25310:34209,dir,BL21_AI_CP047231,4530535,4530564,30
377073,A01492:10:HNH2FDMXX:1:1134:5981:20149,dir,BL21_AI_CP047231,4530537,4530564,28


In [17]:
bbbb.shape

(348566, 6)